<h1>Project_Desciption</h1>

In [1]:
# A large technology have been hacked! Luckily their engineers have grabbed valuable data about the hacks, including 
# information like session time,locations, wpm typing speed, etc. The engineer relates to you what she has
# been able to figure out so far, she has been able to grab meta data of each session that the hackers used to 
# connect to their servers. These are the features of the data:

#     'Session_Connection_Time': How long the session lasted in minutes
#     'Bytes Transferred': Number of MB transferred during session
#     'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
#     'Servers_Corrupted': Number of server corrupted during the attack
#     'Pages_Corrupted': Number of pages illegally accessed
#     'Location': Location attack came from (Probably useless because the hackers used VPNs)
#     'WPM_Typing_Speed': Their estimated typing speed based on session logs.
        
# The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but
# they aren't very sure if the third hacker was involved or not. We want to figure out whether or not the third 
# suspect had anything to do with the attacks, or was it just two hackers?

# One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have 
# roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each
# should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this 
# is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.

In [2]:
# Import the findspark module and initialize it with the specified Spark path
import findspark
findspark.init('/home/mina/python-spark/spark-3.4.0-bin-hadoop3/')

# Import the pyspark module and the SparkSession class
import pyspark
from pyspark.sql import SparkSession

# Create a Spark session with the application name 'hack_data'.
spark = SparkSession.builder.appName('hack_data').getOrCreate()

23/09/13 18:37:26 WARN Utils: Your hostname, mina-VirtualBox resolves to a loopback address: 127.0.1.1; using 192.168.1.143 instead (on interface enp0s3)
23/09/13 18:37:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/13 18:37:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read a CSV file named 'hack_data.csv' into a DataFrame
# The 'inferSchema=True' option infers data types for columns, and 'header=True' treats the first row as column names
dataset = spark.read.csv('hack_data.csv', header=True, inferSchema=True)

# Print the schema of the dataset
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [4]:
# Generate summary statistics for the columns in the 'dataset' DataFrame.
# 'dataset.summary()' computes common summary statistics like count, mean, stddev, min, max, etc.
dataset.summary().show()

23/09/13 18:37:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    25%|           

In [5]:
# Retrieve the first row of the 'dataset' DataFrame
dataset.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [10]:
# Retrieve the list of column names present in the 'dataset' DataFrame.
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [7]:
# Define a list of column names that represent features of interest.
feature_cols = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [8]:
# Import the necessary modules for creating VectorAssembly
from pyspark.ml.feature import VectorAssembler

# Create a VectorAssembler to assemble selected columns into a feature vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol='Features')

# Transform the DataFrame using the VectorAssembler to add the 'Features' column
output_dataset = assembler.transform(dataset)

# Print the schema of the dataset
output_dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- Features: vector (nullable = true)



In [9]:
# Import the StandardScaler class from the PySpark MLlib library
from pyspark.ml.feature import StandardScaler

# Create an instance of the StandardScaler class
# - 'inputCol' specifies the input column to scale ('Features' in this case)
# - 'outputCol' specifies the name of the output scaled column ('Features_scaled' in this case)
scaler = StandardScaler(inputCol='Features', 
                        outputCol='Features_scaled')

# Fit the scaler to the 'output_dataset'
# This computes summary statistics and prepares the scaler for transformation
scaler_fit_db = scaler.fit(output_dataset)

# Transform the 'output_dataset' using the fitted scaler
# This scales the 'features' column and adds a new column 'features_scaled'
scaler_trans_db = scaler_fit_db.transform(output_dataset)
scaler_trans_db.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, Features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), Features_scaled=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963]))]

<h2>Time to find out whether its 2 or 3!</h2>

In [14]:
# Import the KMeans class from the PySpark MLlib library
from pyspark.ml.clustering import KMeans

# Create a KMeans model with 'Features_scaled' as the feature column and 'k' clusters set to 2
kmeans1 = KMeans(featuresCol='Features_scaled' , k = 3 )

# Create another KMeans model with 'Features_scaled' as the feature column and 'k' clusters set to 3
kmeans2 = KMeans(featuresCol='Features_scaled' , k = 2 )

# Fit the first KMeans model (kmeans1) to the 'scaler_trans_db' dataset
model_1 = kmeans1.fit(scaler_trans_db)

# Fit the second KMeans model (kmeans2) to the 'scaler_trans_db' dataset
model_2 = kmeans2.fit(scaler_trans_db)

# Get the training cost (inertia) for the first KMeans model.
# The training cost is a measure of the sum of squared distances from each point to its assigned cluster center
model_1.summary.trainingCost

434.1492898715821

In [18]:
# Get the training cost (inertia) for the second K-Means model (model_2)
# The training cost is a measure of the sum of squared distances from each point to its assigned cluster center
training_cost_2 = model_2.summary.trainingCost
model_2.summary.trainingCost

601.7707512676691

<h3>Not much to be gained from trainingCosts , after all, we would expect that as K increases, the trainingCost decreases. We could however continue the analysis by seeing the drop from K=3 to K=4 to check if the clustering favors even or odd numbers. This won't be substantial, but its worth a look:</h3>

In [23]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='Features_scaled',k=k)
    model = kmeans.fit(scaler_trans_db)
    trainingCost = model.summary.trainingCost
    print("With K={}".format(k))
    print("Within Set Sum of Squared Errors = " + str(trainingCost))
    print('--'*30)

With K=2
Within Set Sum of Squared Errors = 601.7707512676691
------------------------------------------------------------
With K=3
Within Set Sum of Squared Errors = 434.1492898715821
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = 267.1336116887894
------------------------------------------------------------
With K=5
Within Set Sum of Squared Errors = 245.4269716116671
------------------------------------------------------------
With K=6
Within Set Sum of Squared Errors = 231.65828275358692
------------------------------------------------------------
With K=7
Within Set Sum of Squared Errors = 210.1500778742552
------------------------------------------------------------
With K=8
Within Set Sum of Squared Errors = 203.23698492146255
------------------------------------------------------------


<h3>Nothing definitive can be said with the above, but the last key fact that the engineer mentioned was that the attacks should be evenly numbered between the hackers! Let's check with the transform and prediction columns that result form this!</h3>

In [15]:
# Use the first K-Means model (model_1) to make predictions on the 'scaler_trans_db' dataset
# The resulting DataFrame contains a 'prediction' column indicating the cluster assignment
pre_column = model_1.transform(scaler_trans_db).select('prediction')

# Group the data by the 'prediction' column and count the number of data points in each cluster
pre_column.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [17]:
# Use the second K-Means model (model_2) to make predictions on the 'scaler_trans_db' dataset
# The resulting DataFrame contains a 'prediction' column indicating the cluster assignment
pre_column = model_2.transform(scaler_trans_db).select('prediction')

# Group the data by the 'prediction' column and count the number of data points in each cluster
pre_column.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



<h2>Finally it has been solved! It was 2 hackers. In fact, our clustering algorithm created two equally sized clusters with K=2, no way that is a coincidence!</h2>